In [32]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from torchvision.models import VGG16_Weights, ResNet50_Weights


In [33]:
# Load VGG model
def load_vgg_model(path, num_classes):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
    for name, param in model.named_parameters():
        if "28" in name or "29" in name:
            param.requires_grad = True
        else:
            param.requires_grad = False
    model.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(25088, 512),
        nn.Linear(512, 256),
        nn.Dropout(0.5),
        nn.Linear(256, num_classes)
    )
    model.load_state_dict(torch.load(path, map_location=device))
    model = model.to(device)
    model.eval()
    return model

In [34]:
# Load ResNet model
def load_resnet_model(path, num_classes):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
    for name, param in model.named_parameters():
        if "layer4" in name or "fc" in name:
            param.requires_grad = True
        else:
            param.requires_grad = False 
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 512),
        nn.Linear(512, 256),
        nn.Dropout(0.5),
        nn.Linear(256, num_classes)
    )
    model.load_state_dict(torch.load(path, map_location=device))
    model = model.to(device)
    model.eval()
    return model

In [35]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    # transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) 
])

In [36]:
# Ganti dengan kelas dataset kamu
classes = ['Senang', 'Takut']  # contoh

In [37]:
num_classes = len(classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [38]:
model_vgg = load_vgg_model('vgg_emotion_model6.pth', num_classes)
model_resnet = load_resnet_model('resnet_emotion_model5.pth', num_classes)

C:\Users\user\AppData\Local\Temp\ipykernel_87232\3597473485.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path, map_location=device))

In [39]:
# Prediction function
def predict(image_path):
    image = Image.open(image_path).convert('RGB')
    img_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        # VGG prediction
        out_vgg = model_vgg(img_tensor)
        prob_vgg = F.softmax(out_vgg, dim=1)
        conf_vgg, pred_vgg = torch.max(prob_vgg, 1)
        
        # ResNet prediction
        out_resnet = model_resnet(img_tensor)
        prob_resnet = F.softmax(out_resnet, dim=1)
        conf_resnet, pred_resnet = torch.max(prob_resnet, 1)
        
    return (classes[pred_vgg.item()], conf_vgg.item()), (classes[pred_resnet.item()], conf_resnet.item())

In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

def open_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        img = Image.open(file_path).resize((256, 256))
        img = ImageTk.PhotoImage(img)
        panel.configure(image=img)
        panel.image = img

        (vgg_pred, vgg_conf), (resnet_pred, resnet_conf) = predict(file_path)

        label_result1.config(text=f"VGG16 Prediction: {vgg_pred} ({vgg_conf*100:.2f}%)")
        label_result2.config(text=f"ResNet50 Prediction: {resnet_pred} ({resnet_conf*100:.2f}%)")

root = tk.Tk()
root.title("Emotion Detection: VGG16 vs ResNet50")
root.geometry("500x650")
root.configure(bg="#f0f2f5")  

# Frame for styling
frame = tk.Frame(root, bg="white", bd=2, relief="groove")
frame.pack(padx=20, pady=20)

title_label = tk.Label(frame, text="Emotion Detector", font=("Segoe UI", 20, "bold"), bg="white", fg="#333")
title_label.pack(pady=10)

btn = tk.Button(frame, text="Select Image", command=open_file, font=("Segoe UI", 12, "bold"), bg="#4CAF50", fg="white", activebackground="#45a049", padx=10, pady=5, bd=0, relief="ridge", cursor="hand2")
btn.pack(pady=10)

placeholder = Image.open("placeholder.png").resize((256, 256))
placeholder_img = ImageTk.PhotoImage(placeholder)

panel = tk.Label(frame, image=placeholder_img, bg="white", bd=2, relief="solid")
panel.image = placeholder_img  
panel.pack(pady=10)

label_result1 = tk.Label(frame, text="VGG16 Prediction:", font=("Segoe UI", 14), bg="white", fg="#555")
label_result1.pack(pady=5)

label_result2 = tk.Label(frame, text="ResNet50 Prediction:", font=("Segoe UI", 14), bg="white", fg="#555")
label_result2.pack(pady=5)

footer = tk.Label(root, text="Built by Calvin Rooyen using PyTorch and Tkinter", font=("Segoe UI", 9), bg="#f0f2f5", fg="#999")
footer.pack(pady=10)

root.mainloop()